In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

In [10]:
options = Options()
options.add_argument("--start-maximized")  # Cho dễ debug
# options.add_argument("--headless")  # Nếu muốn chạy không mở cửa sổ trình duyệt
driver = webdriver.Chrome(options=options)

In [5]:
# === Hàm scroll đến cuối trang ===
def scroll_to_bottom(driver, pause_time=1.5):
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause_time)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [6]:
def switch_to_list_view(driver):
    try:
        # Đợi SVG "list view" hiện ra
        svg = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'svg[data-qa="list_view"]'))
        )
        button = svg.find_element(By.XPATH, "./ancestor::button")
        button.click()
        print("✅ Đã chuyển sang List View")
        time.sleep(2)
    except Exception as e:
        print("⚠️ Không thể chuyển sang List View:", e)

In [7]:
def get_detail_urls_in_price_range(driver, min_price, max_price):
    url = f"https://www.bluenile.com/diamond-search?CaratFrom=0.05&Color=K,J,I,H,G,F,E,D&PriceFrom={min_price}&PriceTo={max_price}&Clarity=SI2,SI1,VS2,VS1,VVS2,VVS1,IF,FL&Cut=Good,Very+Good,Ideal,AstorIdeal&resultsView=List"
    driver.get(url)
    time.sleep(3)

    # switch_to_list_view(driver)
    scroll_to_bottom(driver)

    # Lấy các thẻ chứa link chi tiết mẫu
    links = driver.find_elements(By.XPATH, "//a[contains(@href, '/diamond-details/')]")
    detail_urls = set([link.get_attribute("href") for link in links])
    print(f"🔍 Tìm thấy {len(detail_urls)} mẫu từ ${min_price} đến ${max_price}")
    return list(detail_urls)

In [ ]:
# === Crawl toàn bộ các khoảng giá ===
all_detail_urls = []

for price_min in range(4000, 10001, 50):  # Từ 4000 đến 10000, bước 50
    price_max = price_min + 49
    try:
        urls = get_detail_urls_in_price_range(driver, price_min, price_max)
        all_detail_urls.extend(urls)
        time.sleep(2)
    except Exception as e:
        print(f"❌ Lỗi khi xử lý khoảng giá {price_min}-{price_max}: {e}")

# === Lưu vào file CSV ===
df = pd.DataFrame({'url': list(set(all_detail_urls))})  # Xóa trùng lặp
df.to_csv("diamond_urls.csv", index=False)
print(f"\n✅ Đã lưu {len(df)} URL vào 'diamond_urls.csv'")

# === Đóng trình duyệt ===
driver.quit()

In [11]:
url = f"https://www.bluenile.com/diamond-search?CaratFrom=0.05&Color=K,J,I,H,G,F,E,D&PriceFrom={4000}&PriceTo={4050}&Clarity=SI2,SI1,VS2,VS1,VVS2,VVS1,IF,FL&Cut=Good,Very+Good,Ideal,AstorIdeal&resultsView=List"
driver.get(url)
time.sleep(3)

scroll_to_bottom(driver)

In [10]:
get_detail_urls_in_price_range(driver,4000, 4050)

🔍 Tìm thấy 89 mẫu từ $4000 đến $4050


['https://www.bluenile.com/diamond-details/24304437',
 'https://www.bluenile.com/diamond-details/11509822',
 'https://www.bluenile.com/diamond-details/21458395',
 'https://www.bluenile.com/diamond-details/23881246',
 'https://www.bluenile.com/diamond-details/24459561',
 'https://www.bluenile.com/diamond-details/24661398',
 'https://www.bluenile.com/diamond-details/24881193',
 'https://www.bluenile.com/diamond-details/24856441',
 'https://www.bluenile.com/diamond-details/24816215',
 'https://www.bluenile.com/diamond-details/24937707',
 'https://www.bluenile.com/diamond-details/23847113',
 'https://www.bluenile.com/diamond-details/24395187',
 'https://www.bluenile.com/diamond-details/23546762',
 'https://www.bluenile.com/diamond-details/24956308',
 'https://www.bluenile.com/diamond-details/16857531',
 'https://www.bluenile.com/diamond-details/18792846',
 'https://www.bluenile.com/diamond-details/17985962',
 'https://www.bluenile.com/diamond-details/22951346',
 'https://www.bluenile.com/d

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# === 1. Khởi tạo trình duyệt ===
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")  # phóng to trình duyệt
# options.add_argument("--headless") 
driver = webdriver.Chrome(options=options)

# === 2. Truy cập trang web ===
url = "https://www.bluenile.com/diamond-details/100142"  # Thay bằng URL sản phẩm thực tế
driver.get(url)

# === 3. Nhấn nút "Show More" nếu có ===
try:
    show_more_btn = WebDriverWait(driver, 2).until(
        EC.element_to_be_clickable((By.XPATH, "//div[@data-qa='show-more']"))
    )
    driver.execute_script("arguments[0].click();", show_more_btn)
    # time.sleep(1)  # đợi 1 giây để phần mở rộng hiện ra
except:
    print("Không tìm thấy nút Show More, bỏ qua...")

# === 4. Hàm lấy dữ liệu từ data-qa bắt đầu bằng tiền tố ===
def get_text_by_dataqa_prefix(prefix):
    try:
        element = driver.find_element(By.XPATH, f"//div[starts-with(@data-qa, '{prefix}')]")
        return element.text.strip()
    except:
        return ""
def get_text_by_dataqa_prefix2(prefix):
    try:
        element = driver.find_element(By.XPATH, f"//div[starts-with(@dataqa, '{prefix}')]")
        return element.text.strip()
    except:
        return ""
# === 5. Thu thập dữ liệu ===
data = {
    
    "carat": get_text_by_dataqa_prefix("CaratWeight-"),
    "cut": get_text_by_dataqa_prefix("Cut-"),
    "color": get_text_by_dataqa_prefix("Color-"),
    "clarity": get_text_by_dataqa_prefix("Clarity-"),
    "depth": get_text_by_dataqa_prefix("depthPrecentage-"),
    "table": get_text_by_dataqa_prefix("tablePrecentage-"),
    "price": get_text_by_dataqa_prefix("price"),
    "measurements": get_text_by_dataqa_prefix2("Measurements-"),
}

# === 6. In ra kết quả ===
print("\n=== Thông tin sản phẩm ===")
for key, value in data.items():
    print(f"{key}: {value}")

# === 7. Đóng trình duyệt sau khi xong ===
driver.quit()


Không tìm thấy nút Show More, bỏ qua...

=== Thông tin sản phẩm ===
carat: 
cut: 
color: 
clarity: 
depth: 
table: 
price: 
measurements: 


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# === Cấu hình Selenium ===
options = Options()
options.add_argument("--start-maximized")  # Mở trình duyệt ở chế độ toàn màn hình
driver = webdriver.Chrome(options=options)  # Khởi tạo trình điều khiển Chrome

# === Đọc các URL chi tiết đã lưu từ file CSV ===
df = pd.read_csv("diamond_urls.csv")  # Đọc danh sách URL đã crawl được
urls = df["url"].tolist()  # Lưu vào danh sách
urls = urls[0:3000]

# === Danh sách để lưu thông tin chi tiết kim cương ===
diamond_data = []

# === Lặp qua từng URL sản phẩm ===
for i, url in enumerate(urls):
    try:
        driver.get(url)  # Mở trang chi tiết kim cương

       

        # === Bấm nút "Show More" nếu có ===
        try:
            show_more_btn = WebDriverWait(driver, 2).until(
                EC.element_to_be_clickable((By.XPATH, "//div[@data-qa='show-more']"))
            )
            driver.execute_script("arguments[0].click();", show_more_btn)
            
        except:
            print("Không tìm thấy nút Show More, bỏ qua...")

        # === 4. Hàm lấy dữ liệu từ data-qa bắt đầu bằng tiền tố ===
        def get_text_by_dataqa_prefix(prefix):
            try:
                element = driver.find_element(By.XPATH, f"//div[starts-with(@data-qa, '{prefix}')]")
                return element.text.strip()
            except:
                return ""
        def get_text_by_dataqa_prefix2(prefix):
            try:
                element = driver.find_element(By.XPATH, f"//div[starts-with(@dataqa, '{prefix}')]")
                return element.text.strip()
            except:
                return ""
        # === 5. Thu thập dữ liệu ===
        data = {
            
            "carat": get_text_by_dataqa_prefix("CaratWeight-"),
            "cut": get_text_by_dataqa_prefix("Cut-"),
            "color": get_text_by_dataqa_prefix("Color-"),
            "clarity": get_text_by_dataqa_prefix("Clarity-"),
            "depth": get_text_by_dataqa_prefix("depthPrecentage-"),
            "table": get_text_by_dataqa_prefix("tablePrecentage-"),
            "price": get_text_by_dataqa_prefix("price"),
            "measurements": get_text_by_dataqa_prefix2("Measurements-"),
        }

        diamond_data.append(data)  # Thêm dữ liệu mẫu vào danh sách
        print(f"[{i+1}/{len(urls)}] ✅ Đã lấy dữ liệu từ: {url}")  # In thông báo

        time.sleep(1)  # Nghỉ một chút để tránh bị chặn

    except Exception as e:
        # Nếu có lỗi khi truy cập hoặc trích xuất dữ liệu
        print(f"[{i+1}] ❌ Lỗi khi lấy dữ liệu từ {url}: {e}")

[1/3000] ✅ Đã lấy dữ liệu từ: https://www.bluenile.com/diamond-details/100142
[2/3000] ✅ Đã lấy dữ liệu từ: https://www.bluenile.com/diamond-details/10079462
[3/3000] ✅ Đã lấy dữ liệu từ: https://www.bluenile.com/diamond-details/10120732
[4/3000] ✅ Đã lấy dữ liệu từ: https://www.bluenile.com/diamond-details/10121753
[5/3000] ✅ Đã lấy dữ liệu từ: https://www.bluenile.com/diamond-details/10134539
[6/3000] ✅ Đã lấy dữ liệu từ: https://www.bluenile.com/diamond-details/10268281
[7/3000] ✅ Đã lấy dữ liệu từ: https://www.bluenile.com/diamond-details/10271184


KeyboardInterrupt: 

In [ ]:
df_detail = pd.DataFrame(diamond_data)  # Chuyển danh sách thành DataFrame
df_detail.to_csv("diamond_details.csv", index=False)  # Xuất ra file CSV
print(f"\n✅ Đã lưu {len(df_detail)} mẫu vào 'diamond_details.csv'")

# === Đóng trình duyệt sau khi hoàn tất ===
driver.quit()

In [1]:
import os
print(os.cpu_count())  # Ví dụ: 8


16
